KAYNAK :

1-) https://pysource.com/2020/04/02/train-yolo-to-detect-a-custom-object-online-with-free-gpu/                                        
2-) https://github.com/AlexeyAB/darknet#how-to-train-to-detect-your-custom-objects                        
3-) https://github.com/AlexeyAB/darknet/wiki/CFG-Parameters-in-the-%5Bnet%5D-section                     
4-) https://github.com/AlexeyAB/darknet/wiki/CFG-Parameters-in-the-different-layers

**Connect google drive**

In [ ]:
# Üstteki sekmeden Düzenle > Not defteri Ayarları na girin. Açılan pencereden Donanım hızlandırıcıyı "GPU" olarak seçin ve kaydete tıklayın.
# Bize tahsis edilen ekran kartını kontrol edelim. Ekran kartı modelini ve bize ayrılan ram miktarını görebilirsniz.
# ! işareti ile linux komutlarını çalıştırıyoruz. Default olarak ise "/content" dizini içerisinde başlıyor. Bu ortamda çalışırken dosyalarımızı genellikle buraya yükleriz.
!nvidia-smi

Sat Nov 28 11:00:40 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Google drive mızı colab ortamına bağlamak için aşağıdaki işlemleri gerçekleştirmemiz lazım

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')  # size bir link verecektir. Gmail hesabına girip, gerekli izinleri verdikten sonra size bir kod verecektir. Verilen kodu aşağıda açılacak olan kutucuğa yapıştırıp entere basın.
!ln -s /content/gdrive/My\ Drive/ /mydrive   # Drive mıza "/mydrive" diyerek ulaşacağız. Çünkü bu yolu bağladık. Default olarak "/content/gdrive/My\ Drive/" şeklinde ulaşılıyor.
!ls /mydrive  # drive daki verilerimizi listeler. Kontrol etmek için yapılan bir kod parçası



**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet # Verilerimizi Darknet frameworku üzerinde eğiteceğiz. Bunu çalıştırdak sonra ilgili dizinde "darknet" klasörü oluşacaktır. 

Cloning into 'darknet'...
remote: Enumerating objects: 14479, done.
remote: Total 14479 (delta 0), reused 0 (delta 0), pack-reused 14479
Receiving objects: 100% (14479/14479), 13.16 MiB | 22.81 MiB/s, done.
Resolving deltas: 100% (9849/9849), done.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
# Biz şuan "/content" içerisindeyiz. İndirdiğimiz darknet frameworku bu dizine kaydedildi.
# cd deyip darknet klasörüne giriyoruz. burada "makefile" dosyamız var. Aşağıdaki kodlar (!sed) bu makefile içindeki değerleri değiştirmeye yarıyor. 
# "make" komutu makefile dosyasına göre dosyaları birbirine bağlayan bir araçtır. Gerekli ayarları yapmak için aşağıdaki kodları  çalıştırın. 
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!make

**3) Configure Darknet network for training YOLO V3**

In [ ]:
# darknet klasörü içerisindeydik. Bu klasör içerisinde ".cfg" dosyalarının bulunduğu cfg klasörümüz var.
#  Biz kendi konfigrasyon ayarlarımızı yapcağımız için cfg klasörü içerisinden "yolov3.cfg" adlı dosyayı yine cfg dizinine "yolov3_training.cfg" adıyla klonluyoruz
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
#Kendi ayarlarımızı yapmak için "yolov3_training.cfg" içerisinde bazı bilgileri değiştirmemiz gerekiyor
# batch -> her döngüde işlenecek resim sayısını belirtir
# subdivisions -> batch, bu kadar çok "bloğa" bölünmüştür. Bir bloğun görüntüleri gpu üzerinde paralel olarak işlenir.
# max_batches -> toplam iterasyon sayımızı belirler.
# classes yazan yere class(sınıf) sayımızı giriyoruz. Örneğin biz sadece drone tespit edeceğimizden ilgili yerlere "1" yazdık 
# filters ayarını ise şu formüle göre yazın -> (classes+5)*3 . Bizim 1 sınıfımız olduğundan (5+1)*3= 18 yazdık. 
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 30000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Daha önceden "/mydrive" yolunu drive'mızın yolu olduğunu söylemiştik. Aşağıdaki komut drive'a "yolov3" adında klasör oluşturacaktır
# Eğer daha önceden oluşturmuşsanız çalıştırsanız da çalıştırmasanızda sorun olmayacaktır
# Bu klasörü oluşturmak önemlidir.Çünkü eğitim esnasında veya bsonunda çıkan ağırlıklarımızı drive'a kayededeceğiz. Kayıt için ileride "/mydrive/yolov3" yolunu vereceğiz

Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

In [ ]:
# darknet dizini içerisindeydik.

# İlk kod darknet/data dizini çerisine "obj.names" adında bir dosya oluşturacak, içerisine "Drone" yazdıracaktır
# "obj.names" dosyası bizim nesne takibindeki sınıflarımızın isimlerini tutatan dosyadır.

# ikinci kod ise darknet/data dizini çerisine "obj.data" adında bir dosya oluşturacaktır. Bu dosyayanın içerisinde aşağıdaki bilgileri kaydecektir.
# "obj.data" dosyasında: 
#    "classes= 1" sınıf sayısı belirtir.
#    "train  = data/train.txt" eğitim verilerinin tutulduğu dosyayının yolunu veriyoruz. "train.txt" ise eğitim için ayrılan resimlerin dizinlerini tutan dosyadır.
#    "valid  = data/test.txt"  test verilerinin tutulduğu dosyayının yolunu veriyoruz. "test.txt" ise test için ayrılan resimlerin dizinlerini tutan dosyadır.
#    "backup = /mydrive/yolov3" ağırlıkların kaydedileceği yolu belirliyoruz. Biz Drive'mıza kaydedeceğimiz yolu verdik. Drive'mızdaki yolov3 klasörüne kaydedilecek.

# üçüncü kod ise "data" dizinine "obj" klasörünü oluşturuyor. Bu "obj" klasörüne ileride eğitim ve test için hazırladğımız resimlerimizi ve txt dosyalarını koyacağız.

!echo "Drone" > data/obj.names 
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [ ]:
# Darknetin öneceden oluşturulmuş ağırlıklarını indiriyoruz.
# Bu modelin içerisine eğitimdeyken değişen ağırlıklarımızı kaydediyoruz ve eğitim bittiğinde çıkan ağırlıkları kaydediyoruz

!wget https://pjreddie.com/media/files/darknet53.conv.74

In [ ]:
# OPSİYONEL !!!
# Daha önce kaydetmiş olduğunuz ağırlıklardan devam etmek istiyorsanız, 
# aşağıdaki örnekteki gibi kendinize göre ayarlayıp Drive'da kayıtlı olan ağırlıkları darknet klasörü içerisine kopyalayabilrisiniz.
#!cp /mydrive/yolov3/yolov3_training_12000.weights ./    #example 1
#!cp /mydrive/yolov3/yolov3_training_last.weights ./     #example 2

**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
# Hala darknet dizini içerisindeyiz.
# Eğitim için hazırlanan resim ve txt dosyalarını rarlayıp "images.rar" adında Drivemıza yükledikten sonra,
# unrar deyip "x" den sonra drive'mızın yolunu verip "-d" den sonra ise buradaki darknet dizini içerisindeki "data/obj" dizinini veriyoruz.
# Yani Drive'daki rar içerisindekileri "data" klasörü içerisindeki "obj" klasörünün içerisine çıkarıyoruz.

!unrar x /mydrive/Colab\ Notebooks/images.rar -d data/obj

NOT : Benim sıkıştırdığım dosyalar "images" klasörü altında geldi. Bu yüzden benim resimlerim "data/obj/images/" diznin içerisinde saklanıyor. İlerleyen aşamalarda ben "data/obj/images/" yolunu kullanacağım. Siz de gerekli kontrolleri yaptıktan sonra kendi yolunuza göre aşağıdaki kodları değiştirin

In [ ]:
# OPSİYONEL! Eğer nesne takibinde tek bir sınıf kullanılıyorsa ve hatalar varsa !!
# Eğerki resimlerin açıklamalarının (txt dosyalarını) içerisinde kordinatları verirken farklı sınıflar vermişseniz onları tek sınıfa göre düzeltir
# Örneğin   1 0.1 0.1 0.3 0.3 -> 0 0.1 0.1 0.3 0.3 
#           5 0.4 0.4 0.6 0.6 -> 0 0.4 0.4 0.6 0.6  olarak değiştirir. Yani başındaki sayıyı 0 yapar. Çünkü bizim tek sınıfımız bu yüzden 0 olmalı.
# Dikkat!  "txt_file_paths = glob.glob(r"data/obj/images/*.txt") " satırına dikkat edin. Buradaki yol sizin resimlerinizin olduğu yol olsun.


import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/images/*.txt")   # "data/obj/images/" yolunu kontrol edin.
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
           # print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text+'\n')
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

In [ ]:
# Hala darknet dizini içerisindeyiz.
# Bu kod ilgili dizindeki her bir resmin dizini+isimi+uzantısı olacak şekilde listesini çıkarıyor 

import glob
images_list = glob.glob("data/obj/images/*.jpg")  # "data/obj/images/" yolunu kontrol edin.
print(images_list)
print(len(images_list))

In [ ]:
# Hala darknet dizini içerisindeyiz.
# Yukarıdaki listeyi darknet klasörü içerisindeki "data" klasörüne "train.txt" olarak yazıdırıyoruz. yani train.txt içerisinde resimlerin yolları mevcut.
# Darknet eğitimde kullanacağı resimleri ve açıklamalarını bu "train.txt" içerisindeki resimlerin yolarından(path) seçip kullanıyor.
# !! Hatırlayacak olursanız "data" dizini içerisinde "obj.data" dosyasını oluştrurken "train.txt" nin yolunu belirtmiştik 

file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

Eğer ki resimlerinizin bir kısmıını test verisi olarak ayarlayacaksınız aşağıdaki kodu çalıştırın.

In [ ]:
# OPSİYONEL!!!
# Hala darknet dizini içerisindeyiz.

import glob 

# Fotoğrafların yüzde kaçını test verisi olarak kullanacaksınız aşağıdaki değeri ona göre değiştirin.
percentage_test = 15;

# "darknet/data" dizinin içersine train.txt ve test.txt oluşturuyoruz.
file_train = open('data/train.txt', 'w')  
file_test = open('data/test.txt', 'w')

# Ayrıştırma işlemini başlatıyor , train.txt ve test.txt içerisine kaydediyor
counter = 1  
index_test = round(100 / percentage_test)  
for i in range(len(images_list)):
    if counter == index_test:
        counter = 1
        file_test.write(images_list[i] + "\n")
    else:
        file_train.write(images_list[i] + "\n")
        counter = counter + 1
file_train.close() 
file_test.close()

**6) Start the training**

In [ ]:
# İlgili parametreleri girerek Eğitime başlıyoruz. 

!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show -map

In [ ]:
# OPSİYONEL!!!
# Daha öncedeki ağırlık dosyanızdan devam etmek istiyorsanız "yolov3.conv.53" yerine "*.weights" uzantılı dosyanızın yolu ile değiştirin. 

#!./darknet detector train data/obj.data cfg/yolov3_training.cfg yolov3_training_last.weights -dont_show -map